In [2]:
!pip install gigachain faiss-cpu sentence-transformers sentencepiece rank_bm25 datasets --quiet


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import datasets
from langchain.docstore.base import Document
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

## Данные

Загрузим Question Answering датасет на русском языке - [SberQuAD](https://huggingface.co/datasets/sberquad).

Каждая запись содержит несколько полей, из которых нам нужны будут только 2:
- **question** - неструктурированный запрос или обычный вопрос;
- **contex** - релевантный этому вопросу параграф из базы знаний (пассаж).

In [5]:
ds = datasets.load_dataset("sberquad")
ds["train"][0]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

{'id': 62310,
 'title': 'SberChallenge',
 'context': 'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.',
 'question': 'чем представлены органические остатки?',
 'answers': {'text': ['известковыми выделениями сине-зелёных водорослей'],
  'answer_start': [109]}}

In [6]:
validation_ds = ds["validation"]
documents = [
    Document(page_content=context) for context in set(validation_ds["context"])
]

## Ретривал

Наша задача - среди всех возможных пассажей найти наиболее подходящий вопросу, чтобы на основе него ГигаЧат смог дать максимально точный ответ.

Для этой задачи существует множество обёрток различных retrieval подходов внутри пакета `langchain.retrievers`. Попробуем найти наиболее точный для наших данных

### Эмбеддинг модель

Попробуем использовать нейросетевой ретривер (эмбеддер) с векторной базой данных.

В качестве эмбеддингов возьмём специализированную мультиязычную модель [intfloat/multilingual-e5-base](https://huggingface.co/intfloat/multilingual-e5-base). Модель требует использовать различные префиксы для запросов и пассажей, поэтому создадим свой класс `HuggingFaceE5Embeddings` под эту логику, немного переопределив стандартный класс `langchain.embeddings.HuggingFaceEmbeddings`.

Вы можете попробовать использовать любую энкодер модель для вашей задачи. Для русского языка существует открытый бенчмарк энкодеров предложений [avidale/encodechka](https://github.com/avidale/encodechka#лидерборд), который содержит отдельный лидерборд для ранжирования. Также можете посмотреть специализированный мультиязычный лидерборд [MTEB](https://huggingface.co/spaces/mteb/leaderboard)

In [7]:
from typing import List, Coroutine, Any


class HuggingFaceE5Embeddings(HuggingFaceEmbeddings):
    def embed_query(self, text: str) -> List[float]:
        text = f"query: {text}"
        return super().embed_query(text)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        texts = [f"passage: {text}" for text in texts]
        return super().embed_documents(texts)

    async def aembed_query(self, text: str) -> Coroutine[Any, Any, List[float]]:
        text = f"query: {text}"
        return await super().aembed_query(text)

    async def aembed_documents(
        self, texts: List[str]
    ) -> Coroutine[Any, Any, List[List[float]]]:
        texts = [f"passage: {text}" for text in texts]
        return await super().aembed_documents(texts)

In [8]:
embedding = HuggingFaceE5Embeddings(model_name="intfloat/multilingual-e5-base")

.gitattributes:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

onnx/config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device

А в качестве векторной базы данных используем [faiss](https://faiss.ai/), как самое популярное и простое решение для подобных задач. При создании экземпляра векторной БД для каждого документа считается его векторное представление, поиск по которому и происходит при каждом запросе - вся база данных ранжируется по степени релевантности входному запросу. В результате `embedding_retriever` вернёт `k = 5` самых подходящих пассажей.

GigaChain, как и Langchain, предоставляет большое количество интерфейсов под все самые популярные векторные БД, полный список можно найти на [странице документации](https://python.langchain.com/docs/integrations/vectorstores).

In [7]:
faiss_db = FAISS.from_documents(documents, embedding=embedding)

In [8]:
embedding_retriever = faiss_db.as_retriever(search_kwargs={"k": 5})

In [9]:
validation_ds = validation_ds.map(
    lambda x: {
        "embedding_retrieved": [
            passage.page_content
            for passage in embedding_retriever.get_relevant_documents(x["question"])
        ]
    }
)
validation_ds

Map: 100%|██████████| 5036/5036 [08:03<00:00, 10.42 examples/s]


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'embedding_retrieved'],
    num_rows: 5036
})

В качестве целевой метрики используем точность нахождения целевого пассажа среди топ-5 отранжированных

In [9]:
def acc_top(dataset: datasets.Dataset, right_column: str, answer_column: str) -> float:
    temp_dataset = dataset.map(
        lambda x: {"is_right_retrieved": x[right_column] in x[answer_column]}
    )
    return sum(temp_dataset["is_right_retrieved"]) / len(temp_dataset)

In [10]:
acc_top(validation_ds, "context", "embedding_retrieved")

Map:   0%|          | 0/5036 [00:00<?, ? examples/s]

KeyError: 'embedding_retrieved'

### BM25

In [13]:
from langchain.retrievers import EnsembleRetriever, BM25Retriever

UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 459: character maps to <undefined>

Теперь попробуем оценить качество другого ретривала - [BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)

Попробуйте использовать токенизацию предложений на русском от библиотеки [natasha/razdel](https://github.com/natasha/razdel#usage), [aatimofeev/spacy_russian_tokenizer](https://github.com/aatimofeev/spacy_russian_tokenizer#usage) или [Koziev/rutokenizer](https://github.com/Koziev/rutokenizer#примеры) - они могут улучшить точность поиска на ваших данных

In [11]:
import string


def tokenize(s: str) -> list[str]:
    """Очень простая функция разбития предложения на слова"""
    return s.lower().translate(str.maketrans("", "", string.punctuation)).split(" ")

In [13]:
bm25_retriever = BM25Retriever.from_documents(
    documents=documents,
    preprocess_func=tokenize,
    k=5,
)

In [14]:
validation_ds = validation_ds.map(
    lambda x: {
        "bm25_retrieved": [
            passage.page_content
            for passage in bm25_retriever.get_relevant_documents(x["question"])
        ]
    }
)
validation_ds

Map: 100%|██████████| 5036/5036 [06:34<00:00, 12.75 examples/s]


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'embedding_retrieved', 'bm25_retrieved'],
    num_rows: 5036
})

In [15]:
acc_top(validation_ds, "context", "bm25_retrieved")

Map: 100%|██████████| 5036/5036 [00:01<00:00, 3617.90 examples/s]


0.9197776012708498

### Ансамбль

А теперь попробуем увеличить точность поиска, объединив работу двух предыдущих ретривалов алгоритмом [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf). Для него есть удобная обёртка `langchain.retrievers.EnsembleRetriever`

In [16]:
embedding_retriever = faiss_db.as_retriever(search_kwargs={"k": 2})
bm25_retriever = BM25Retriever.from_documents(
    documents=documents,
    preprocess_func=tokenize,
    k=3,
)

In [17]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[embedding_retriever, bm25_retriever],
    weights=[0.4, 0.6],
)

In [18]:
validation_ds = validation_ds.map(
    lambda x: {
        "retrieved": [
            passage.page_content
            for passage in ensemble_retriever.get_relevant_documents(x["question"])
        ]
    }
)
validation_ds

Map: 100%|██████████| 5036/5036 [20:28<00:00,  4.10 examples/s]


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'embedding_retrieved', 'bm25_retrieved', 'retrieved'],
    num_rows: 5036
})

In [19]:
acc_top(validation_ds, "context", "retrieved")

Map: 100%|██████████| 5036/5036 [00:01<00:00, 3121.68 examples/s]


0.9690230341540905

## E2E решение

На улучшенном решении уже можно строить вопросно-ответную систему с достаточно хорошей точностью ответов. Ограничимся простой цепочкой `langchain.chains.RetrievalQA`

In [20]:
from langchain.chains import RetrievalQA
from langchain.llms.gigachat import GigaChat

In [21]:
llm = GigaChat(credentials=...)

In [22]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=ensemble_retriever,
    return_source_documents=True,
)

In [23]:
qa.invoke({"query": "Что такое вода?"})

{'query': 'Что такое вода?',
 'result': 'Вода — это бинарное неорганическое соединение, состоящее из двух атомов водорода и одного атома кислорода, соединенных ковалентной связью. Она является прозрачной жидкостью без цвета, запаха и вкуса при нормальных условиях. Вода может существовать в различных состояниях, включая жидкое, твердое и газообразное. Она играет важную роль в жизни на Земле и является основой для существования всех живых организмов.',
 'source_documents': [Document(page_content='На самом деле из-за низкого давления вода не может существовать в жидком состоянии на большей части (около 70 %) поверхности Марса. Вода в состоянии льда была обнаружена в марсианском грунте космическим аппаратом НАСА Феникс . В то же время собранные марсоходами Спирит и Opportunity геологические данные позволяют предположить, что в далёком прошлом вода покрывала значительную часть поверхности Марса. Наблюдения в течение последнего десятилетия позволили обнаружить в некоторых местах на поверхнос